In [1]:
import os
import time
import json
import requests
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI
from langdetect import detect


In [2]:
API_KEY = "8J6pTdfaGgA5r193UVLsBshUspqwNpal42Jse1aHaok1cWNTLpRkJQQJ99BDACYeBjFXJ3w3AAABACOGLa23"

In [3]:

client = AzureOpenAI(
  azure_endpoint = "https://ai-bcds.openai.azure.com/",
  api_key= API_KEY,
  api_version="2024-05-01-preview"
)

assistant = client.beta.assistants.create(
    model="gpt-4o-mini",  # replace with model deployment name
    name="Assistant Group N (R)",
    instructions="Nunca uses mais do que 700 caracteres. Tu és um perito que trabalha na Fidelidade em investimentos a prazo. Lembra-te que estas a falar com pessoas que não têm formação financeira. Simplifica a linguagem e por vezes faz um pouco de literacia financeira. Recomenda sempre produtos da Fidelidade sempre que for um investimento que faça sentido outros não serão permitidos. Informações não disponiveis nos ficheiros está fora do teu ambito como tal diz que não sabes responder a essa questão. Recorre sempre a fontes disponiveis nos ficheiros e faz citações.",
    tools=[{"type":"file_search"}]
    ,tool_resources={"file_search":{"vector_store_ids":["vs_2oe1I9LEsJcvJEKmbJnBuLwk"]},"code_interpreter":{"file_ids":["assistant-WuJZaGApRBgPZGvfPJwTBF","assistant-WQbe2oYNiunshZFamDkAMe","assistant-WA9mo2ooGuCakdtLYe7S5Y","assistant-VmGKDJtKWW8mLr7RsNDHJ4","assistant-VcoN7vmxYPZCBfSTaFsEQ3","assistant-TaHujXgSvATXXH1V5gBTDN","assistant-NMXvUzELtRbwtQMoTXeApo","assistant-LvBZjKCKvtwUvCCTgpfWim","assistant-GWrPgzCPp8si1AH2yKKJBB","assistant-F5nrFbNEjGxEqeSCmkdLAg","assistant-EwB2EpAtrmQVUrtxvg79HB","assistant-BFYazbjKgte4W5u9Jgrs6b","assistant-96mBrzvMJbaTjLYcCEG3S2","assistant-8phYrqvFLjksiThSKqSgkq","assistant-7GHqFyWe29skpMH8Ug4QoZ","assistant-6zJbn35vHos3RRzqJUiVMQ","assistant-5dMu4U3q2VUkjhP4LvhAGb","assistant-4hduKqwh6y7AWcUohbxMC3","assistant-426n1B453C2wqH2AnDc2mh","assistant-3UFjejDd1iMmYhBuj2bgi8"]}},
    temperature=0.05,
    top_p=0.9
)

### 1. One question prompt

In [4]:
question = "O Fidelidade Savings é mais indicado para que perfil de risco?"


In [5]:
from IPython.display import display, Markdown
import re
# Create a thread
thread = client.beta.threads.create()

# Add a user question to the thread
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content=question # Replace this with your prompt
)



# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

# Looping until the run completes or fails
while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )

  for msg in reversed(messages.data):
    cleaned = re.sub(r'【\d+:\d+†source】', '', msg.content[0].text.value)
    display(Markdown(f"**{msg.role}**: {cleaned}"))
elif run.status == 'requires_action':
  # the assistant requires calling some functions
  # and submit the tool outputs back to the run
  pass
else:
  print(run.status)

C:\Users\ruite\AppData\Local\Temp\ipykernel_24776\1691583907.py:4: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\ruite\AppData\Local\Temp\ipykernel_24776\1691583907.py:7: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(
C:\Users\ruite\AppData\Local\Temp\ipykernel_24776\1691583907.py:16: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(
C:\Users\ruite\AppData\Local\Temp\ipykernel_24776\1691583907.py:24: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(
C:\Users\ruite\AppData\Local\Temp\ipykernel_24776\1691583907.py:30: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = client.beta.threads.messages.list(


**user**: O Fidelidade Savings é mais indicado para que perfil de risco?

**assistant**: O Fidelidade Savings é adequado para diferentes perfis de risco, dependendo da opção escolhida:

1. **Opção Seguro**: Ideal para clientes com perfil prudente, que buscam segurança total do capital investido e rendimento garantido (variável). É a opção mais conservadora.

2. **Opção Proteção Ações**: Destinada a clientes com perfil equilibrado, oferece uma garantia de 90% do capital investido, permitindo um maior potencial de ganho, mas com algum risco.

3. **Opção Dinâmico Ações**: Para clientes com perfil dinâmico, que aceitam o risco de não ter garantia do capital, buscando maior rentabilidade através de investimentos em ações.

Assim, o produto se adapta a diferentes necessidades e tolerâncias ao risco.

### 2. Sequential prompt chat

In [6]:
# STEP 2: Function to send a message to the assistant and get a reply
def chat_with_assistant(user_message, thread_id,language):

    # Add a user question to the thread
    client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message # Replace this with your prompt
    )

    if language == 'en':
        run = client.beta.threads.runs.create_and_poll(
        thread_id=thread_id,
        assistant_id=assistant.id,
        additional_instructions='From now on, please respond in English.',

        )
    else:
    # Create a run to process the message
        run = client.beta.threads.runs.create_and_poll(
            thread_id=thread_id,
            assistant_id=assistant.id,
            additional_instructions='From now on, please respond in Portuguese.',

        )

    # Get assistant's response
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    assistant_messages = [msg for msg in messages.data if msg.role == "assistant"]

    msg = ''
    #get the last assistant message
    
    msg+= re.sub(r'【\d+:\d+†source】', '', assistant_messages[0].content[0].text.value)
    return msg


In [7]:
def detect_language(text):
    try:
        lang = detect(text)
        return 'pt' if lang == 'pt' else 'en'
    except:
        return 'pt'

In [ ]:
thread = client.beta.threads.create()

# STEP 3: Chat loop (example)
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]: break

    lang = detect_language(user_input)
    
    display(Markdown(f"> **👤 You:**\n> {user_input.replace('\n', '\n> ')}"))
    print("Assistant is thinking...")
    reply = chat_with_assistant(user_input, thread.id,lang)
    
    display(Markdown(f"> **🤖 Assistant:**\n> {reply.replace('\n', '\n> ')}"))